In [1]:
import cv2
import dlib
import imutils
from imutils import face_utils
import numpy as np
from scipy.spatial import distance as dist
import time
import winsound
from flask import Flask, render_template


app = Flask(__name__)
@app.route('/run_eye_tracking')
def run_eye_tracking():
    def play_alarm():
        #time.sleep(3)
        frequency = 3500  # Set Frequency To 2500 Hertz
        duration = 5000  # Set Duration To 1000 ms == 1 second
        winsound.Beep(frequency, duration)
   

    # Define a function to calculate the eye aspect ratio
    def eye_aspect_ratio(eye):
        A = dist.euclidean(eye[1], eye[5])
        B = dist.euclidean(eye[2], eye[4])
        C = dist.euclidean(eye[0], eye[3])
        ear = (A + B) / (2.0 * C)
        return ear


    
    # Initialize dlib's face detector and create a dictionary to map facial landmark indexes to face parts
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    face_parts = face_utils.FACIAL_LANDMARKS_IDXS
    (left_eye_start, left_eye_end) = face_parts["left_eye"]
    (right_eye_start, right_eye_end) = face_parts["right_eye"]

    # Start the video stream and wait for the camera to warm up
    vs = cv2.VideoCapture(0)
    time.sleep(2.0)

    # Initialize variables for calculating the attention level
    drowsy_counter = 0
    drowsy_threshold = 15
    focused_counter = 0
    focused_threshold = 5
    unfocused_counter = 0
    unfocused_threshold = 20
    max_attention_level = "unfocused"
    max_attention_time = time.time()
    
    eyes_closed = False

    # Loop over frames from the video stream
    while True:
        # Read the next frame from the video stream and resize it
        _, frame = vs.read()
        frame = imutils.resize(frame, width=450)
    
        # Convert the frame to grayscale and detect faces
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        rects = detector(gray, 0)
    
        # Reset the eye positions
        left_eye_position = None
        right_eye_position = None

        # Loop over the face detections
        for rect in rects:
            # Determine the facial landmarks for the face region, then convert the facial landmark (x, y)-coordinates to a NumPy array
            shape = predictor(gray, rect)
            shape = face_utils.shape_to_np(shape)
    
            # Extract the left and right eye coordinates and compute the eye aspect ratio for each eye
            left_eye = shape[left_eye_start:left_eye_end]
            right_eye = shape[right_eye_start:right_eye_end]
            left_eye_ear = eye_aspect_ratio(left_eye)
            right_eye_ear = eye_aspect_ratio(right_eye)

            # Compute the average eye aspect ratio
            avg_ear = (left_eye_ear + right_eye_ear) / 2.0
    
            # Check if the average eye aspect ratio is below the drowsy threshold
            if avg_ear < 0.25:
                drowsy_counter += 1
                focused_counter = 0
                unfocused_counter = 0
            else:
                drowsy_counter = 0
                focused_counter += 1
                # Check if eyes are not in the frame
                if left_eye.shape[0] == 0 or right_eye.shape[0] == 0:
                    unfocused_counter += 1
                else:
                    left_eye_position = left_eye
                    right_eye_position = right_eye

            # Check if the attention level is drowsy
            if drowsy_counter >= drowsy_threshold:
                max_attention_level = "drowsy"
                if drowsy_counter == drowsy_threshold:
                    alarm_start_time = time.time()
                if time.time() - alarm_start_time >= 3 and not eyes_closed:
                    play_alarm()
                    #eyes_closed = True
                    drowsy_counter = 0
            
            # Check if the attention level is unfocused
            if unfocused_counter >= unfocused_threshold:
                max_attention_level = "unattentive"

            # Check if the attention level is focused
            if focused_counter >= focused_threshold:
                max_attention_level = "focused"

	  

     # Update the max attention level
        if time.time() - max_attention_time >= 10:
            print(f"Max attention level for the past 10 Seconds : {max_attention_level}")
            max_attention_level = "unattentive"
            max_attention_time = time.time()

        # Display the frame
        cv2.imshow("Frame", frame)
        key = cv2.waitKey(1) & 0xFF

        # If the `q` key was pressed, break from the loop
        if key == ord("q"):
            break

    # Cleanupq
    cv2.destroyAllWindows()
    #vs.stop()

